In [3]:
import cv2
import time
import pandas as pd
from pynput import keyboard
import numpy as np
from matplotlib import pyplot as plt
import robomaster
from robomaster import robot
from robomaster import camera


## 加速时 
\
恒定功率加速,则牵引力 $ F=\frac{P}{v} $ ,设机车质量为m,阻力为f
$$ \frac{P}{v} - f = ma = m\frac{dv}{dt} $$
得到解为：
$$ \frac{mf^2}{P}t = -\frac{f}{P}v - \ln\left(1 - \frac{f}{P}v\right) $$
\
带入机甲大师s1 $ V_m = 1 m/s $ $ P = 2 W $ $ m = 3.3 kg $ $ f = \frac{P}{V_m} $
$$ 6.6t = -v - \ln\left(1 - v\right)  $$
## 减速时 
\
易知 $ f = 2N $ 则 $ a_d = f/m ≈ 0.6 m/s^2 $
$$ V = V_m - at $$
$$ V = - 0.6t + 1 $$
\
以下为加速和减速时v-t图像

In [ ]:
v1 = np.arange(0, 1, 0.00001)
t1 = (1/6.6)*(-v1-np.log(1-v1))
plt.plot(t1,v1)

t2 = np.arange(0, 1.6, 0.0001)
v2 = -0.6*t2 + 1
plt.plot(t2,v2)


创建对应列表

In [ ]:
v1 = np.arange(0, 1, 0.001)
t1 = (1/6.6)*(-v1-np.log(1-v1))
ac = [0,0.76,0.88,0.94,0.96,1,1]

de = list()
for i in range(0,17):
   de.append(round(-0.6*i/10 + 1,2))
de.append(0), de.append(0)

测量两个间隔0.1s图片间隔角度\
OpenCV的SIFT算法来找到两幅图像中的关键点和描述符，然后使用最近邻匹配来找到良好的特征点对。\
计算得到角度

In [10]:
# 平移对齐图片
def align_images(img1, img2):
    # 将图片转换为灰度图
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 使用 SIFT 特征检测器检测关键点和描述符
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    # 使用 FLANN 匹配器进行特征匹配
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # 选择最佳匹配点
    good = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good.append(m)

    # 获取关键点的坐标
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # 使用 RANSAC 算法估计单应性矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # 对图像进行透视变换
    h, w = img1.shape[:2]
    img1_aligned = cv2.warpPerspective(img1, M, (w, h), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

    return img1_aligned, img2

def calculate_rotation_angle(img1, img2):
    # 读取图片
    # img1 = cv2.imread("train/images/1.jpg")
    # img2 = cv2.imread("train/images/2.jpg")

    # 将图片转换为灰度图
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 使用 SIFT 特征检测器检测关键点和描述符
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    # 使用 FLANN 匹配器进行特征匹配
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # 选择最佳匹配点
    good = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good.append(m)

    # 获取关键点的坐标
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # 使用 RANSAC 算法估计单应性矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # 获取旋转角度
    angle = np.arctan(M[0, 1] / M[0, 0]) * 180 / np.pi
    return angle
    
    
def find_index(lst, target):
    # 使用 lambda 函数计算每个元素与目标数的绝对差值，并返回最小差值的索引
    return min(range(len(lst)), key=lambda i: abs(lst[i] - target))
w_pressed = o_pressed = n_pressed = False
def on_press(key):
    try:
        if key.char == 'w':
            global w_pressed
            w_pressed = True
        elif key.char == 'o':
            global o_pressed
            o_pressed = True
            return False
        elif key.char == 'n':
            global n_pressed
            n_pressed = True
    except AttributeError:
        pass
def on_release(key):
    try:
        if key.char == 'w':
            global w_pressed
            w_pressed = False
        elif key.char == 'o':
            global o_pressed
            o_pressed = False
            return False
        elif key.char == 'n':
            global n_pressed
            n_pressed = False
    except AttributeError:
        pass

img1, img2 = align_images(cv2.imread("train/images/54.jpg"), cv2.imread("train/images/53.jpg"))
angle = calculate_rotation_angle(img1, img2)
cv2.waitKey(0)
cv2.destroyAllWindows()
angle

23.181771406349764

In [ ]:
ep_robot = robot.Robot()
# 路由器组网
ep_robot.initialize(conn_type='sta')
# 直连
# robomaster.config.LOCAL_IP_STR = "192.168.2.1"
# ep_robot.initialize(conn_type='ap')
ep_chassis= ep_robot.chassis
ep_camera = ep_robot.camera
ep_camera.start_video_stream(display=True)
# 开启键盘监听事件
listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()


In [ ]:
spd = 0
img_index = 1
datas = {"image_path":[],"angle":[],"speed":[],"direct":[]}
while not o_pressed:
    img = ep_camera.read_cv2_image()
    # 更新时间速度
    if w_pressed:
        tm = round(find_index(ac,spd)/10 + 0.1,6)
        spd = round(ac[int(10*tm)],6)
    else:
        tm = round(find_index(de,spd)/10 + 0.1,6)
        spd = round(de[int(10*tm)],6)
    # 按住p记录数据
    if n_pressed:
        datas['direct'].append([1,0,0])
        try:
            datas['angle'].append(round(detect_rotation_angle(last_img, img), 2))
        except:
            datas['angle'].append(0)
        image_path = "train/images/"+str(img_index)+".jpg"
        cv2.imwrite(image_path, img)
        datas['image_path'].append(image_path)
        datas['speed'].append(spd)
        img_index += 1
    last_img = img
    print(spd)
    time.sleep(0.1)
d = pd.DataFrame(datas)
d.to_csv('train/dataset.csv', index=False)

ep_robot.close()
listener.stop()